In [17]:
import numpy as np
import pandas as pd
import pyodbc

In [18]:
driver = 'ODBC Driver 17 for SQL Server'
server = 'sqlcat1.toromont.com,1685' 
                      
conn = pyodbc.connect('Driver={};Server={};Trusted_Connection=yes; \
                      ApplicationIntent=ReadOnly;database=Appdb_Equiplink'
                      .format(driver, server))

In [19]:
sql = """
---Jeremy Equipment List---
WITH JWL as (
SELECT [EquipmentId] as ID
,SerialNumber AS SN
,EMT
  FROM [AppDb_Equiplink].[AI].[ConditionSummary]
  WHERE
  CMA = 'Jeremy Froats'
  AND
  [EMT] = 1
)
---Event List under 360 days---
SELECT
         jwl.SN
         ,CASE
         WHEN Type = 1 Then 'SOS'
         WHEN Type = 8 Then 'EventCode'
         WHEN Type = 16 Then 'DiagnosticCode'
         else ''
         End as Type
      ,CONVERT(varchar, EventDate, 1) as Date
	  --,EventDate
      ,[Identifier] as Message
      ,[Description]
      ,[Details]
      ,[SMU]
      ,CASE
         WHEN EH.[Level] = 8 THEN 'LV3'
         WHEN EH.[Level] = 4 THEN 'LV2'
         WHEN EH.[Level] = 2 THEN 'LV1'
         ELSE ''
         END AS Severity
  FROM [AppDb_Equiplink].[AI].[EventHistory] as EH
inner join JWL
       ON EH.EquipmentId = JWL.ID
  WHERE EventDate > DATEADD(day,-5,CURRENT_TIMESTAMP) 
and 
  Type in (1,8,16,256,8192,4294967296)

  order by SN,Type,description,EventDate desc
---Points for the Events:Compare to Health Reference Table---
"""

In [20]:
df = pd.read_sql(sql, conn)
df.head()

,SN,Type,Date,Message,Description,Details,SMU,Severity
0,2T400506,DiagnosticCode,04/09/20,"MID: -1, CID: 5519, FMI: 18, OCC: 5",CID: 5519 / MID: -1,Low - moderate severity (2),NaN,LV2
1,2T400506,EventCode,04/13/20,"MID: 389,EID: 1449,OCC: 1",Transmission/Chassis Control,Low Brake Accumulator Precharge Gas Pressure,NaN,LV3
2,2T400583,,04/13/20,v3.3619423,Dealer Process/Heavy Rents Off Rent Full V4.6 ...,,1984.0,LV3
3,2T400583,EventCode,04/13/20,"ET/PSR Active MID: 27,EID: 764, Severity: 1",Transmission/Chassis,Trip Data Memory Full (ET),1985.0,LV1
4,2T400583,EventCode,04/13/20,"ET/PSR Logged MID: 27,EID: 1668,FMI: ,OCC: 5, ...",Transmission/Chassis,High Center Axle Oil Temperature (ET),1167.0,LV1


# Split

In [21]:
eventcodedf = df[(df.Type=='EventCode') | (df.Type=='DiagnosticCode')]
split = eventcodedf.loc[:, 'Message'].str.split(pat=',', expand=True)

test = (pd.melt(split.reset_index(), id_vars='index').dropna().
 drop(columns='variable').set_index('index')
 .value.str.split(':', expand=True)
 .reset_index()
 .pivot(index='index', columns=0, values=1)
#  .head()
)
#combine columns
test['OCC'] = test['OCC'].fillna(test[' OCC'])
test['FMI'] = test['FMI'].fillna(test[' FMI'])
test['CID'] = test['CID'].fillna(test[' CID'])
test['MID'] = test['MID'].fillna(test['ET/PSR Logged MID']).fillna(test['ET/PSR Active MID'])
#replace ' Severity' with ''
test['OCC'] = test['OCC'].str.replace('Severity$', '', regex=True)
# ' Severity' check
# test[test['OCC'].str.contains('Severity')]
#drop columns
test = test.drop(columns=[' OCC', ' FMI', ' CID', 'ET/PSR Logged MID', 'ET/PSR Active MID'])
#rename ' Severity' to 'SeverityFrmMsg'
test = test.rename(columns={' Severity': 'SeverityFrmMsg'})

In [22]:
# test[test['SeverityFrmMsg'].notnull()]
test.head()

,SeverityFrmMsg,CID,EID,FMI,MID,OCC
index,,,,,,
0,NaN,5519,NaN,18,-1,5
1,NaN,NaN,1449,NaN,389,1
3,1,NaN,764,NaN,27,NaN
4,2,NaN,1668,,27,5
5,1,NaN,47,,27,1


In [27]:
eventcat = pd.concat([eventcodedf, test], axis=1)
# eventcat.loc[[198],]
eventcat.head()

,SN,Type,Date,Message,Description,Details,SMU,Severity,SeverityFrmMsg,CID,EID,FMI,MID,OCC
0,2T400506,DiagnosticCode,04/09/20,"MID: -1, CID: 5519, FMI: 18, OCC: 5",CID: 5519 / MID: -1,Low - moderate severity (2),NaN,LV2,NaN,5519,NaN,18,-1,5
1,2T400506,EventCode,04/13/20,"MID: 389,EID: 1449,OCC: 1",Transmission/Chassis Control,Low Brake Accumulator Precharge Gas Pressure,NaN,LV3,NaN,NaN,1449,NaN,389,1
3,2T400583,EventCode,04/13/20,"ET/PSR Active MID: 27,EID: 764, Severity: 1",Transmission/Chassis,Trip Data Memory Full (ET),1985.0,LV1,1,NaN,764,NaN,27,NaN
4,2T400583,EventCode,04/13/20,"ET/PSR Logged MID: 27,EID: 1668,FMI: ,OCC: 5, ...",Transmission/Chassis,High Center Axle Oil Temperature (ET),1167.0,LV1,2,NaN,1668,,27,5
5,2T400583,EventCode,04/13/20,"ET/PSR Logged MID: 27,EID: 47,FMI: ,OCC: 1, Se...",Transmission/Chassis,Transmission Abuse Warning (ET),1884.0,LV1,1,NaN,47,,27,1


In [28]:
#write csv
eventcat.to_csv('event_test.csv', index=False)

# Draft

In [ ]:
test[test[' CID'].notnull()]

In [ ]:
(pd.melt(split.reset_index(), id_vars='index').dropna().
 drop(columns='variable').set_index('index')
 .value.str.split(':', expand=True)
 .reset_index()
 .pivot(index='index', columns=0, values=1)
 .head()
)

In [ ]:
(pd.melt(split.reset_index(), id_vars='index').dropna()
 .drop(columns='variable').set_index('index')
 .value.str.split(':', expand=True)
 # add strip empty space step
 .reset_index()
 .pivot(index='index', columns=0, values=1)
)

In [159]:
eventcodedf = df[(df.Type=='EventCode') | (df.Type=='DiagnosticCode')]
split = eventcodedf.loc[:, 'Message'].str.split(pat=',', expand=True)
eventcat = pd.concat([eventcodedf, split], axis=1)
eventcat.head()
pd.options.display.max_colwidth = 1000
eventcat
# eventcat.loc[[126],]

,SN,Type,Date,Message,Description,Details,SMU,Severity,0,1,2,3,4
1,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 104",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,MID: 389,EID: 47,OCC: 104,None,None
2,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 103",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,MID: 389,EID: 47,OCC: 103,None,None
3,2T400506,DiagnosticCode,04/09/20,"MID: -1, CID: 5519, FMI: 18, OCC: 5",CID: 5519 / MID: -1,Low - moderate severity (2),NaN,LV2,MID: -1,CID: 5519,FMI: 18,OCC: 5,None
4,2T400506,EventCode,04/13/20,"MID: 389,EID: 1449,OCC: 1",Transmission/Chassis Control,Low Brake Accumulator Precharge Gas Pressure,NaN,LV3,MID: 389,EID: 1449,OCC: 1,None,None
5,2T400517,EventCode,04/08/20,"MID: 390,EID: 1248,OCC: 1",Chassis Control Module,Front Cross Axle Lock Speed Mismatch,NaN,LV1,MID: 390,EID: 1248,OCC: 1,None,None
6,2T400517,EventCode,04/08/20,"MID: 390,EID: 1251,OCC: 1",Chassis Control Module,Interaxle Lock Speed Mismatch,NaN,LV1,MID: 390,EID: 1251,OCC: 1,None,None
7,3T300392,EventCode,04/08/20,"MID: 390,EID: 1388,OCC: 6",Chassis Control Module,Operator Seat Belt Unfastened While Machine is Not Idle,NaN,LV1,MID: 390,EID: 1388,OCC: 6,None,None
8,3T300405,EventCode,04/08/20,"MID: 390,EID: 1533,OCC: 2",Chassis Control Module,Unacceptable Machine Rear Roll Angle,NaN,LV3,MID: 390,EID: 1533,OCC: 2,None,None
9,3T300405,EventCode,04/08/20,"MID: 297,EID: 155,OCC: 2",Transmission,High Torque Converter Oil Temperature,NaN,LV2,MID: 297,EID: 155,OCC: 2,None,None
10,3T300405,EventCode,04/08/20,"MID: 297,EID: 155,OCC: 1",Transmission,High Torque Converter Oil Temperature,NaN,LV2,MID: 297,EID: 155,OCC: 1,None,None


# Second Split

In [24]:
#MID (column 0)
split2 = eventcat.loc[:, 0].str.split(pat=':', expand=True)
#check unique values
for col in split2:
    print(split2[col].unique())
eventcat[['MIDs', 'MID']] = split2
eventcat = eventcat.drop(columns=[0])
eventcat.head()

['MID' 'ET/PSR Logged MID']
[' 389' ' -1' ' 390' ' 297' ' 341' ' 263' ' 272' ' 36' ' 464' ' 432'
 ' 299' ' 391' ' 264' ' 122' ' 82' ' 39' ' 400']


,SN,Type,Date,Message,Description,Details,SMU,Severity,1,2,3,4,MIDs,MID
1,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 104",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,EID: 47,OCC: 104,None,None,MID,389
2,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 103",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,EID: 47,OCC: 103,None,None,MID,389
3,2T400506,DiagnosticCode,04/09/20,"MID: -1, CID: 5519, FMI: 18, OCC: 5",CID: 5519 / MID: -1,Low - moderate severity (2),NaN,LV2,CID: 5519,FMI: 18,OCC: 5,None,MID,-1
4,2T400506,EventCode,04/13/20,"MID: 389,EID: 1449,OCC: 1",Transmission/Chassis Control,Low Brake Accumulator Precharge Gas Pressure,NaN,LV3,EID: 1449,OCC: 1,None,None,MID,389
5,2T400517,EventCode,04/08/20,"MID: 390,EID: 1248,OCC: 1",Chassis Control Module,Front Cross Axle Lock Speed Mismatch,NaN,LV1,EID: 1248,OCC: 1,None,None,MID,390


In [25]:
#column 1
split2 = eventcat.loc[:, 1].str.split(pat=':', expand=True)
#replace ' CID' to 'CID'
split2[0] = split2[0].replace({' CID': 'CID'})
#check unique values
#for col in split2:
#    print(split2[col].unique())
eventcat[['EIDs', 'EIDValues']] = split2
eventcat = eventcat.drop(columns=[1])
eventcat.head()
pvt = eventcat.pivot(columns='EIDs', values='EIDValues')
eventcat = pd.concat([eventcat, pvt], axis=1).drop(columns=['MIDs', 'EIDs', 'EIDValues'])
#eventcat.info()

In [26]:
#column 2
split3 = eventcat.loc[:, 2].str.split(pat=':', expand=True)
#replace
split3[0] = split3[0].replace({' FMI': 'FMI', ' Severity': 'Severity_1', 'Severity': 'Severity_1'})
#check unique values
#for col in split3:
#    print(split3[col].unique())
#Concat
eventcat[['occs', 'occValues']] = split3
eventcat = eventcat.drop(columns=[2])
eventcat.head()
pvt = eventcat.pivot(columns='occs', values='occValues')
eventcat = pd.concat([eventcat, pvt], axis=1).drop(columns=['occs', 'occValues'])
#eventcat.head()

In [27]:
#column 3
split4 = eventcat.loc[:, 3].str.split(pat=':', expand=True)
#replace
split4[0] = split4[0].replace({' OCC': 'OCC_1', 'OCC': 'OCC_1', ' Severity ': 'Severity_2', None: 'None'})
#check unique values
#for col in split4:
#    print(split4[col].unique())
split4 = split4.drop(columns=[2])
#Concat
eventcat[['occs', 'occValues']] = split4
eventcat = eventcat.drop(columns=[3])
eventcat.head()
pvt = eventcat.pivot(columns='occs', values='occValues')
eventcat = pd.concat([eventcat, pvt], axis=1).drop(columns=['occs', 'occValues', 'None'])

In [28]:
#column 4
split5 = eventcat.loc[:, 4].str.split(pat=':', expand=True)
#replace
split5[0] = split5[0].replace({' Severity': 'Severity_3', None: 'None_1'})
#check unique values
#for col in split5:
#    print(split5[col].unique())
#Concat
eventcat[['occs', 'occValues']] = split5
eventcat = eventcat.drop(columns=[4])
pvt = eventcat.pivot(columns='occs', values='occValues')
eventcat = pd.concat([eventcat, pvt], axis=1).drop(columns=['occs', 'occValues', 'None_1'])
eventcat.head()

,SN,Type,Date,Message,Description,Details,SMU,Severity,MID,CID,EID,FMI,OCC,OCC_1,Severity_3
1,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 104",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,389,NaN,47,NaN,104,NaN,NaN
2,2T300763,EventCode,04/08/20,"MID: 389,EID: 47,OCC: 103",Transmission/Chassis Control,Transmission Abuse Warning,NaN,LV1,389,NaN,47,NaN,103,NaN,NaN
3,2T400506,DiagnosticCode,04/09/20,"MID: -1, CID: 5519, FMI: 18, OCC: 5",CID: 5519 / MID: -1,Low - moderate severity (2),NaN,LV2,-1,5519,NaN,18,NaN,5,NaN
4,2T400506,EventCode,04/13/20,"MID: 389,EID: 1449,OCC: 1",Transmission/Chassis Control,Low Brake Accumulator Precharge Gas Pressure,NaN,LV3,389,NaN,1449,NaN,1,NaN,NaN
5,2T400517,EventCode,04/08/20,"MID: 390,EID: 1248,OCC: 1",Chassis Control Module,Front Cross Axle Lock Speed Mismatch,NaN,LV1,390,NaN,1248,NaN,1,NaN,NaN


In [17]:
#write csv
eventcat.to_csv('event_test.csv', index=False)

# Combine duplicated columns

In [ ]:
type(str(eventcat['OCC_1'].iloc[1])) #.str.cat(eventcat['OCC_1'],sep="")
eventcat['OCC_1']

In [23]:
def comb(row):
    if row['OCC'].notnull():
        return row['OCC']
    elif row['OCC'].isnull():
        return row['OCC1']

eventcat['occnew'] = eventcat.apply(comb, axis=1)
eventcat.head()

SyntaxError: invalid syntax (<ipython-input-23-d62437cb0091>, line 4)